## In dit script gaan we het CCC model en het DCC model schatten met student-t verdeling

Om de cel hier onder te runnen moet er een package geinstalleerd worden via anaconda prompt. Deze package zorgt ervoor dat we functies uit andere scripts kunnen gerbuiken in dit script.
<break>
Tik het volgende in anaconda prompt:
<br \>
**pip install ipynb**


In [1]:
%%capture
import pandas as pd
import numpy as np
from numpy.linalg import inv
from numpy.linalg import det
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
from ipynb.fs.full.Dataset import getdata
from ipynb.fs.full.Dataset import getreturns
from scipy.optimize import minimize
from scipy.special import gammaln
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = getreturns()
mInSampleReturns = df.loc[:"2015"]

Om de CCC de DCC modellen te schatten kan de log-likelihood opgedeeld worden in 2 delen.
- Schat univariate GARCH(1,1) modellen los van elkaar met student-t verdeling.
- schat daarna de correlatie term met de parameters die je in stap 1 hebt berekend. 


<break>
De univariate student-t distribution van de GARCH modellen zien er als volgt uit:

$$    LL=\sum_{t=2}^T\log\left( \Gamma\left(\frac{\nu+1}{2}\right)\right)-\log\left( \Gamma\left(\frac{\nu}{2}\right)\right)-\frac{1}{2}\log\left(\pi\left(\nu-2\right)\sigma_t^2\right)-\frac{\nu+1}{2}\log\left(1+\frac{y_t^2}{(\nu-2)\sigma_t^2}\right)$$
Waarbij $\sigma_t^2$ is gedefinieerd als
$$\sigma_t^2=\omega+\alpha y_t^2 + \beta\sigma_{t-1}^2$$
In de cel hieronder is een functie geschreven om een losse GARCH(1,1) model te schatten. De parameters die we hierbij moeten schatten zijn:

- [$\omega,\alpha,\beta,\nu$]

In [4]:
def Parameters_GARCH_Student(vReturns):
    dOmega = 0.1
    dAlpha = 0.1
    dBeta = 0.8
    dNu = 4.2
    vTheta = np.array([dOmega, dAlpha, dBeta, dNu])
    
    def LL_GARCH_Student(vTheta,returns):
        dOmega = vTheta[0]
        dAlpha = vTheta[1]
        dBeta  = vTheta[2]
        dNu = vTheta[3]
        iT=len(returns)
        vH=np.zeros(iT)
        
        for t in range(iT):
            if t == 0:
                vH[t] = np.var(returns) 
            else:
                vH[t] = dOmega + dAlpha*returns[t-1]**2 + dBeta * vH[t-1]    
        
        vLogPdf = gammaln( (dNu+1)/2 )-gammaln( dNu/2 ) - 0.5*np.log( np.pi*(dNu-2)*vH[1:] ) \
                - 0.5 * (dNu+1) * np.log( 1 + ( vReturns[1:]**2 / ( (dNu-2)*vH[1:] ) ) )
        return -np.sum(vLogPdf)
    
    def Optimizer(returns, initials, function, bnds):
        result = minimize(function, initials, args=(returns), \
                          options ={'eps':1e-09, 'disp': True, 'maxiter':200}, method='SLSQP',bounds=bnds)
        return result
    bounds = ((0, 1), (0, 1), (0, 1), (2.1,50))
    result=Optimizer(vReturns, vTheta, LL_GARCH_Student, bounds)
    return result.x, -result.fun, result.success

In [5]:
(parameter1_Stud,likelihood1_Stud,success1_Stud)=Parameters_GARCH_Student(np.array(mInSampleReturns.iloc[:,0]))
print(parameter1_Stud)
print(likelihood1_Stud)
print(success1_Stud)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2492.9007501403294
            Iterations: 16
            Function evaluations: 107
            Gradient evaluations: 16
[ 0.04129691  0.09040065  0.89042978  6.9246734 ]
-2492.9007501403294
True


In [6]:
(parameter2_Stud,likelihood2_Stud,success2_Stud)=Parameters_GARCH_Student(np.array(mInSampleReturns.iloc[:,1]))
print(parameter2_Stud)
print(likelihood2_Stud)
print(success2_Stud)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2217.307544488237
            Iterations: 16
            Function evaluations: 108
            Gradient evaluations: 16
[ 0.02623554  0.09365917  0.89023515  6.98573207]
-2217.307544488237
True


In [7]:
def ComputeSigma2GARCH(vTheta1,vTheta2,mReturns):
    iT = mReturns.shape[1]
    iDimension = mReturns.shape[0]
    mH = np.zeros((iDimension,iT))
    dOmega1 = vTheta1[0]
    dOmega2 = vTheta2[0]
    dAlpha1 = vTheta1[1]
    dAlpha2 = vTheta2[1]
    dBeta1 = vTheta1[2]
    dBeta2 = vTheta2[2]
    for t in range(iT):
        if t==0:
            mH[0,t]=np.var(mReturns[0,:])
            mH[1,t]=np.var(mReturns[1,:])
        else:
            mH[0,t]=dOmega1 + dAlpha1 * mReturns[0,t-1]**2 + dBeta1 * mH[0,t-1]
            mH[1,t]=dOmega2 + dAlpha2 * mReturns[1,t-1]**2 + dBeta2 * mH[1,t-1]
    return mH

In [8]:
mSigma2=ComputeSigma2GARCH(parameter1_Stud,parameter2_Stud,np.array(mInSampleReturns).T)

Hieronder maximaliseren we de log-likelihood van de correlatie term van het CCC model. 
De log-likelihood van de correlatie term ziet er als volgt uit:
$$\sum_{t=2}^{T}\left(\log\Gamma\left(\frac{\nu+n}{2}\right)-\log\Gamma\left(\frac{\nu}{2}\right)-\frac{n}{2}\log(\pi(\nu-2))-\frac{1}{2}\log\left(|D_t|^2\right)-\frac{1}{2}\log\left(|R_t|\right)-\left(\frac{\nu+n}{2}\right)\log\left(1+\frac{\epsilon_t'R_t^{-1}\epsilon_t}{\nu-2}\right)\right)$$
- waarbij R is gedefinieerd als:
$$R=\begin{pmatrix} 1 & \rho_{12} \\ \rho_{12} & 1 \end{pmatrix}$$
- $\epsilon_t$ als:
$$\epsilon_t=\begin{pmatrix} \epsilon_{1t} \\ \epsilon_{2t}\end{pmatrix}$$
- $\epsilon_{it}$ is verkregen door:
$$\epsilon_{it}=\frac{y_{it}}{\sigma_{it}}$$
De log-likelihood willen we maximaliseren wrt $\rho_{12}$ en $\nu$

In [9]:
def Parameters_Correlation_StudentCCC(mReturns,mH):
    dRho12 = 0.2
    dNu = 4.2
    vTheta = np.array([dRho12, dNu])
    
    def LL_Correlation_StudentCCC(vTheta,mReturns,mH):
        dRho12 = vTheta[0]
        dNu = vTheta[1]
        iT = mReturns.shape[1]
        iDimension = len(mReturns)
        mEpsilon = mReturns/np.sqrt(mH)
        mR = np.ones((iDimension,iDimension))
        mR[1,0] = dRho12
        mR[0,1] = dRho12
        
        dSum=0
        
        for t in range(1,iT):
            mD_t=np.diag(np.sqrt(mH[:,t]))
            vEpsilon_t = mEpsilon[:,t]
            vEpsilon_t = vEpsilon_t.reshape((iDimension,1))
            dLogLikelihood = gammaln((iDimension+dNu)/2) - gammaln(dNu/2) - iDimension/2 * np.log(np.pi*(dNu-2)) \
                            -0.5*np.log( det(mD_t)**2 ) - 0.5 * np.log( det(mR) ) - ( (dNu+iDimension)/2 ) * \
                            np.log(1+ ( ( vEpsilon_t.T @ inv(mR) @vEpsilon_t ) / (dNu-2) ) )
            dSum += np.asscalar(dLogLikelihood)
  
        return -dSum
    
    def Optimizer(returns, mH, initials, function, bnds):
        result = minimize(function, initials, args=(returns,mH), \
                          options ={'eps':1e-09, 'disp': True, 'maxiter':200}, method='SLSQP',bounds=bnds)
        return result
    bounds = ((-0.9999999, 0.9999999),(2.1,50))
    result=Optimizer(mReturns, mH, vTheta, LL_Correlation_StudentCCC, bounds)
    return result.x, -result.fun, result.success

In [10]:
(vParameters_CCC,LL_Correlation_CCC,success_Corr_CCC)=Parameters_Correlation_StudentCCC(np.array(mInSampleReturns).T,mSigma2)
print(vParameters_CCC)
print(LL_Correlation_CCC)
print(success_Corr_CCC)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3215.4318508079887
            Iterations: 29
            Function evaluations: 129
            Gradient evaluations: 28
[ 0.92582585  7.03458001]
-3215.4318508079887
True


Hier gaan we de correlatie term van het DCC model schatten. De log-likelihood ziet er als volgt uit:
$$\sum_{t=2}^{T}\left(\log\Gamma\left(\frac{\nu+n}{2}\right)-\log\Gamma\left(\frac{\nu}{2}\right)-\frac{n}{2}\log(\pi(\nu-2))-\frac{1}{2}\log\left(|D_t|^2\right)-\frac{1}{2}\log\left(|R_t|\right)-\left(\frac{\nu+n}{2}\right)\log\left(1+\frac{\epsilon_t'R_t^{-1}\epsilon_t}{\nu-2}\right)\right)$$
- Waarbij $R_t$ is gedefinieerd als:
$$ R_t=\text{diag}(Q_t)^{\frac{-1}{2}}Q_t\text{diag}(Q_t)^{\frac{-1}{2}}$$
- Q_t is gedefinieerd als:
$$Q_t=(1-A-B)S + A\epsilon_{t-1}\epsilon_{t-1}' + BQ_{t-1}$$
-S is gedefinieerd als:
$$S=\frac{1}{T}\sum_{t=1}^{T}\epsilon_t\epsilon_t'$$
verder gebruik ik als initialisatie van $Q_1=S$
<break>
- De $diag(Q_t)^\frac{-1}{2}$ is een diagonaal matrix waarbij de elementen op de diagonaal matrix hetzelfde zijn als die van $Q_t$ alleen dan nog in de wortel genomen
- In andere woorden:
$$diag(Q_t)^{-\frac{1}{2}}=\begin{pmatrix} \sqrt(q_{11t}) & 0 \\ 0 & \sqrt(q_{22t}) \end{pmatrix}^{-1}, Q_t=\begin{pmatrix} q_{11t} & q_{12t} \\ q_{21t} & q_{22t} \end{pmatrix} $$
- De log-likelihood wordt gemaximaliseerd wrt A, B, $\nu$, waarbij A en B getallen zijn die positief moeten zijn en $\nu>2$

In [11]:
def Parameters_Correlation_StudentDCC(mReturns,mH):
    dA = 0.2
    dB = 0.7
    dNu = 4.2
    vTheta = np.array([dA,dB,dNu])
    
    def LL_Correlation_StudentDCC(vTheta,mReturns,mH):
        dA = vTheta[0]
        dB = vTheta[1]
        dNu = vTheta[2]
        iT = mReturns.shape[1]
        iDimension = len(mReturns)
        mEpsilon = mReturns/np.sqrt(mH)

        mS_sum = np.zeros((iDimension,iDimension))
        for i in range(iT):
            vEpsilon_t = mEpsilon[:,i]
            vEpsilon_t = vEpsilon_t.reshape((iDimension,1))
            mS_sum += vEpsilon_t @ vEpsilon_t.T
        
        mS = mS_sum / iT
        mQ_old=np.copy(mS)
        
        dSum=0
        for t in range(1,iT):
            mD_t=np.diag(np.sqrt(mH[:,t]))
            vEpsilon_lag=mEpsilon[:,t-1]
            vEpsilon_lag=vEpsilon_lag.reshape((iDimension,1))
            vEpsilon=mEpsilon[:,t]
            vEpsilon=vEpsilon.reshape((iDimension,1))
            
            mQ = (1-dA-dB) * mS + dA * vEpsilon_lag @ vEpsilon_lag.T + dB * mQ_old 
            mQ_diagonal = np.sqrt(np.diagonal(mQ))
            mR_t= inv(np.diag(mQ_diagonal)) @ mQ @ inv(np.diag(mQ_diagonal)) 
            
            dLogLikelihood = gammaln((iDimension+dNu)/2) - gammaln(dNu/2) - iDimension/2 * np.log(np.pi*(dNu-2)) \
                            -0.5*np.log( det(mD_t)**2 ) - 0.5 * np.log( det(mR_t) ) - ( (dNu+iDimension)/2 ) * \
                            np.log(1+ ( ( vEpsilon.T @ inv(mR_t) @vEpsilon ) / (dNu-2) ) )
            dSum += np.asscalar(dLogLikelihood)
            mQ_old=mQ 
        return -dSum
    
    def Optimizer(returns, mH, initials, function, bnds):
        result = minimize(function, initials, args=(returns,mH), \
                          options ={'eps':1e-09, 'disp': True, 'maxiter':200}, method='SLSQP',bounds=bnds)
        return result
    bounds = ((0.0001, 0.9999), (0.0001,0.9999), (2.1,50))
    result=Optimizer(mReturns, mH, vTheta, LL_Correlation_StudentDCC, bounds)
    return result.x, -result.fun, result.success

In [12]:
(vParametersDCC,LL_Correlation_DCC,success_Corr_DCC)=Parameters_Correlation_StudentDCC(np.array(mInSampleReturns).T,mSigma2)
print(vParametersDCC)
print(LL_Correlation_DCC)
print(success_Corr_DCC)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3173.9248196202416
            Iterations: 15
            Function evaluations: 99
            Gradient evaluations: 15
[ 0.05806218  0.90943246  7.52368661]
-3173.9248196202416
True


Hier gaan we de correlatie term van het DCC model schatten. De log-likelihood ziet er als volgt uit:
$$\sum_{t=2}^{T}\left(\log\Gamma\left(\frac{\nu+n}{2}\right)-\log\Gamma\left(\frac{\nu}{2}\right)-\frac{n}{2}\log(\pi(\nu-2))-\frac{1}{2}\log\left(|D_t|^2\right)-\frac{1}{2}\log\left(|R_t|\right)-\left(\frac{\nu+n}{2}\right)\log\left(1+\frac{\epsilon_t'R_t^{-1}\epsilon_t}{\nu-2}\right)\right)$$
- Waarbij $R_t$ is gedefinieerd als:
$$ R_t=\text{diag}(Q_t)^{\frac{-1}{2}}Q_t\text{diag}(Q_t)^{\frac{-1}{2}}$$
- Q_t is gedefinieerd als:
$$Q_t=(1-A-B)S -G\bar{N} + A\epsilon_{t-1}\epsilon_{t-1}' + Gn_{t-1}n_{t-1}' + BQ_{t-1}$$
- S is gedefinieerd als:
$$S=\frac{1}{T}\sum_{t=1}^{T}\epsilon_t\epsilon_t'$$
verder gebruik ik als initialisatie voor $Q_1=S$
<break>
- n_{t-1} is gedefinieerd als: 
$$
n_{t-1}=\mathbb{1}(\epsilon_{t-1})\odot \epsilon_{t-1}
\\
\mathbb{1}=\begin{cases}
1&\text{if $\epsilon_{t-1}<0$}\\
0&\text{otherwise}\\
\end{cases}
$$
- Hierbij is $\bar{N}$ gedefinieerd als:
$$\bar{N}=\frac{1}{T}\sum_{t=1}^{T}n_tn_t'
$$
Verder gebruiken we als initialisatie $n_1n_1'=\bar{N}$
- De $diag(Q_t)^\frac{-1}{2}$ is een diagonaal matrix waarbij de elementen op de diagonaal matrix hetzelfde zijn als die van $Q_t$ alleen dan nog in de wortel genomen
- In andere woorden:
$$diag(Q_t)^{-\frac{1}{2}}=\begin{pmatrix} \sqrt(q_{11t}) & 0 \\ 0 & \sqrt(q_{22t}) \end{pmatrix}^{-1}, Q_t=\begin{pmatrix} q_{11t} & q_{12t} \\ q_{21t} & q_{22t} \end{pmatrix} $$
- De log-likelihood wordt gemaximaliseerd wrt A, B, G, $\nu$ waarbij A, B en G getallen zijn en die positief moeten zijn en $\nu>2$


In [13]:
def Parameters_Correlation_StudentADCC(mReturns,mH):
    dA = 0.1
    dB = 0.7
    dG = 0.1
    dNu = 4.2
    vTheta = np.array([dA,dB,dG,dNu])
    
    def LL_Correlation_StudentADCC(vTheta,mReturns,mH):
        dA = vTheta[0]
        dB = vTheta[1]
        dG = vTheta[2]
        dNu = vTheta[3]
        iT = mReturns.shape[1]
        iDimension = len(mReturns)
        mEpsilon = mReturns/np.sqrt(mH)
        mZeros = np.zeros(mReturns.shape)
        mDummy=(mEpsilon<mZeros)+mZeros
        mN=mEpsilon*mDummy        

        mS_sum = np.zeros((iDimension,iDimension))
        mN_sum = np.zeros((iDimension,iDimension))
        for i in range(iT):
            vN_t=mN[:,i]
            vN_t=vN_t.reshape((iDimension,1))
            mN_sum += vN_t @ vN_t.T            
            vEpsilon_t = mEpsilon[:,i]
            vEpsilon_t = vEpsilon_t.reshape((iDimension,1))
            mS_sum += vEpsilon_t @ vEpsilon_t.T
        
        mS = mS_sum / iT
        mN_bar=mN_sum / iT
        mQ_old=np.copy(mS)
        
        dSum=0
        for t in range(1,iT):
            mD_t=np.diag(np.sqrt(mH[:,t]))
            vEpsilon_lag=mEpsilon[:,t-1]
            vEpsilon_lag=vEpsilon_lag.reshape((iDimension,1))
            vEpsilon=mEpsilon[:,t]
            vEpsilon=vEpsilon.reshape((iDimension,1))
            vN_lag=mN[:,t-1]
            vN_lag=vN_lag.reshape((iDimension,1))            
            
            mQ = (1-dA-dB) * mS - dG * mN_bar + dA * vEpsilon_lag @ vEpsilon_lag.T + dG * vN_lag @ vN_lag.T + dB * mQ_old 
            mQ_diagonal = np.sqrt(np.diagonal(mQ))
            mR_t= inv(np.diag(mQ_diagonal)) @ mQ @ inv(np.diag(mQ_diagonal)) 
            
            dLogLikelihood = gammaln((iDimension+dNu)/2) - gammaln(dNu/2) - iDimension/2 * np.log(np.pi*(dNu-2)) \
                            -0.5*np.log( det(mD_t)**2 ) - 0.5 * np.log( det(mR_t) ) - ( (dNu+iDimension)/2 ) * \
                            np.log(1+ ( ( vEpsilon.T @ inv(mR_t) @vEpsilon ) / (dNu-2) ) )
            dSum += np.asscalar(dLogLikelihood)
            mQ_old=mQ   
        return -dSum
    
    def Optimizer(returns, mH, initials, function, bnds):
        result = minimize(function, initials, args=(returns,mH), \
                          options ={'eps':1e-09, 'disp': True, 'maxiter':200}, method='SLSQP',bounds=bnds)
        return result
    bounds = ((0.0001, 0.9999), (0.0001,0.9999), (0.0001,0.9999),(2.1,50))
    result=Optimizer(mReturns, mH, vTheta, LL_Correlation_StudentADCC, bounds)
    return result.x, -result.fun, result.success

In [14]:
(vParametersADCC,LL_Correlation_ADCC,success_Corr_ADCC)=Parameters_Correlation_StudentADCC(np.array(mInSampleReturns).T,mSigma2)
print(vParametersADCC)
print(LL_Correlation_ADCC)
print(success_Corr_ADCC)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3173.1956888161494
            Iterations: 22
            Function evaluations: 163
            Gradient evaluations: 22
[ 0.04127358  0.91498665  0.02541066  7.45767264]
-3173.1956888161494
True
